use simplified data 2

data source: ./simplified_data/simplified_data2.csv

data 處理:

data v2 刪掉所有文字input

deleted 回診,西藥

## train 方法:

1個model train所有藥

In [2]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

# custom import 
from utility_file import my_utilities as myutil
from tabulate import tabulate

## Utility function

In [3]:
def ReadData(FILENAME):
    data = pd.read_csv(FILENAME, encoding='ANSI')
    print("in ReadData")
    print("type of data:", type(data))
    print(f'Shape of data = ({data.shape[0]} rows, {data.shape[1]} cols).')
    return data

def SplitXY(data, data_left_bound, label_left_bound, data_right_bound=None, label_right_bound=None):
    """label_left_bound: 第一個藥材的col no."""
    # Body status: 1~3, Diagnosis: 4~7, Symptom: 11~124
    # Prescription: 125~226
    # split_X = list(range(0, label_left_bound))
    # split_Y = list(range(label_left_bound, len(data.columns)))
    if label_right_bound is None:
        label_right_bound = len(data.columns)
    if data_right_bound is None:
        data_right_bound = label_left_bound
    X = data.iloc[1:, list(range(data_left_bound, data_right_bound))]
    y = data.iloc[1:, list(range(label_left_bound, label_right_bound))]
    
    # Debug
    print("SplitXY:")
    print(f'Shape of X = ({X.shape[0]} rows, {X.shape[1]} cols). First 10 data of X:')
    print(X.iloc[:10, :10])
    print(f'Shape of y = ({y.shape[0]} rows, {y.shape[1]} cols). First 10 data of y:')
    print(y.iloc[:10, :10])
    return X, y

def SplitNparr(original_arr: np.ndarray, train_portion: float)->tuple:
    data_len =len(original_arr)
    train_len = int(data_len * train_portion)
    indices = list(range(data_len))
    random.shuffle(indices)
    train_idx = indices[:train_len]
    validate_idx = indices[train_len:]
    training_arr  = original_arr[train_idx]
    validation_arr = original_arr[validate_idx]

    return (training_arr,validation_arr)


def Split1Df(data: pd.DataFrame,  train_size: float, random_state: int=None):
    """
    Split the data into training and validation sets.

    Param:
    - data: any data
    - train_size: Proportion of in train set.
    - random_state: Seed 

    return:
    - data_train: data for the training set.
    - data_val: data for the validation set.

    """
    if random_state is None:
        random_state = 42
    random.seed(random_state)
    data_len = len(data)
    val_len = int(data_len * (1-train_size))
    indices = list(range(data_len))
    random.shuffle(indices)
    val_idx = indices[:val_len]
    train_idx = indices[val_len:]


    data_train, data_val = data.loc[train_idx], data.loc[val_idx]
   
    return data_train, data_val

def SplitBothXy_Df(X: pd.DataFrame, y: pd.DataFrame, train_size: float, random_state: int=None):
    """
    Split the data into training and validation sets.

    Param:
    - X: Features 
    - y: Target variable
    - train_size: Proportion of in train set.
    - random_state: Seed 

    return:
    - X_train: Features for the training set.
    - X_val: Features for the validation set.
    - y_train: Target variable for the training set.
    - y_val: Target variable for the validation set.
    """
    if random_state is not None:
        random.seed(random_state)
    data_len = len(X)
    val_len = int(data_len * (1-train_size))
    indices = list(range(data_len))
    random.shuffle(indices)
    val_idx = indices[:val_len]
    train_idx = indices[val_len:]
    print("type of val_idx", type(val_idx))
    print("type of train_idx", type(train_idx))
    print("len of val_idx", len(val_idx))

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert(X_train.shape[1] == X_val.shape[1])
    assert(y_train.shape[1] == y_val.shape[1])
    return X_train, X_val, y_train, y_val


In [4]:
def DeleteMedicine(y):
    threshold = 250
    for col in y.columns:
        # print(f'Number of {col} is {y[col].sum()}.')
        if y[col].sum() < threshold:
            y = y.drop(col, axis=1)
    # Debug
    print(f'DeleteMedicine: shape of y is {y.shape}.')
    return y

## Read Data

In [5]:
data_pd = ReadData("./simplified_data/simplified_data2.csv")

in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).


In [6]:
first_medicine_idx = None
#first_medicine_idx = 113
first_medicane = '麻黃'

for i in range(len(data_pd.columns.tolist())):
    if data_pd.columns.tolist()[i] == first_medicane:
        first_medicine_idx = i
        break
print("first_medicine: ",data_pd.columns[first_medicine_idx])
print("first_medicine_idx: ", first_medicine_idx)
print("should be 麻黃")




first_medicine:  麻黃
first_medicine_idx:  113
should be 麻黃


## 決定要train 多少個藥

In [7]:

num_med_to_train = 3  # train 從第一隻藥數起多少隻
# 
X,y = SplitXY(data_pd, data_left_bound=2,
              data_right_bound=first_medicine_idx, 
              label_left_bound=first_medicine_idx, 
              label_right_bound=first_medicine_idx+num_med_to_train)

train_X, val_X, train_y, val_y = SplitBothXy_Df(X, y, 0.8)

X_np = train_X.values.astype('float64')
X_val_np = val_X.values.astype('float64')
num_col_x = X_np.shape[1]
y_np = train_y.values.astype('float64')

num_medic = train_y.shape[1]



SplitXY:
Shape of X = (796 rows, 111 cols). First 10 data of X:
    肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤  肝炎
1    0    0   0   0     0   0    0   0   0   0
2    0    0   0   0     0   0    0   0   0   0
3    0    0   0   0     0   0    0   0   0   0
4    0    0   0   0     0   0    0   0   0   0
5    0    0   0   0     0   0    0   0   0   0
6    0    0   0   0     0   0    0   0   0   0
7    0    0   0   0     0   0    0   0   0   0
8    0    0   0   0     0   0    0   0   0   0
9    0    0   0   0     0   0    0   0   0   0
10   0    0   0   0     0   0    0   0   0   0
Shape of y = (796 rows, 3 cols). First 10 data of y:
    麻黃  桂枝  荊芥
1    0   1   0
2    0   1   0
3    0   0   0
4    0   1   0
5    0   1   0
6    0   0   0
7    0   1   0
8    0   1   0
9    0   1   0
10   0   1   0
type of val_idx <class 'list'>
type of train_idx <class 'list'>
len of val_idx 159
train_X.shape:  (637, 111)
train_y.shape:  (637, 3)


In [8]:
# debug message 
print("train_X.shape: ", train_X.shape)
print("train_y.shape: ", train_y.shape)

print("X transformed to np array")
print("type of X_np:", type(X_np))
print("shape of X_np:", X_np.shape)
print("type of X_val_np:", type(X_val_np))
print("shape of X_val_np:", X_val_np.shape)

print("number of col in (train) x:",num_col_x)
print("shape of train y:", train_y.shape)

print("y transformed to np array")
print("type of y_np:", type(y_np))
print("shape of y_np:", y_np.shape)

print("no. of medicine in train_y:", num_medic)


X transformed to np array
type of X_np: <class 'numpy.ndarray'>
shape of X_np: (637, 111)
type of X_val_np: <class 'numpy.ndarray'>
shape of X_val_np: (159, 111)
number of col in (train) x: 111
shape of train y: (637, 3)
y transformed to np array
type of y_np: <class 'numpy.ndarray'>
shape of y_np: (637, 3)
no. of medicine in train_y: 3


In [23]:
# debug only

print(tabulate(train_y,  tablefmt='psql', floatfmt=(".0f")))



+-----+---+---+---+
| 715 | 0 | 0 | 0 |
| 423 | 0 | 1 | 0 |
| 591 | 0 | 0 | 0 |
| 569 | 0 | 0 | 0 |
|  60 | 0 | 1 | 0 |
| 665 | 0 | 1 | 0 |
| 314 | 0 | 1 | 0 |
| 266 | 0 | 1 | 0 |
| 442 | 0 | 0 | 0 |
| 751 | 1 | 1 | 0 |
| 400 | 1 | 0 | 0 |
|  21 | 0 | 1 | 0 |
| 589 | 1 | 1 | 0 |
| 744 | 0 | 0 | 0 |
| 170 | 0 | 0 | 0 |
| 233 | 0 | 0 | 0 |
| 232 | 0 | 0 | 0 |
| 137 | 0 | 0 | 0 |
| 151 | 0 | 0 | 0 |
| 219 | 0 | 0 | 0 |
|  38 | 0 | 0 | 0 |
| 640 | 0 | 0 | 0 |
| 609 | 1 | 0 | 1 |
|  33 | 1 | 1 | 0 |
| 726 | 0 | 0 | 0 |
| 158 | 0 | 0 | 0 |
|  86 | 0 | 1 | 0 |
| 627 | 0 | 1 | 0 |
| 639 | 0 | 0 | 0 |
| 115 | 0 | 0 | 0 |
| 196 | 1 | 0 | 0 |
| 269 | 0 | 0 | 0 |
| 599 | 0 | 0 | 0 |
|  13 | 0 | 1 | 0 |
| 728 | 0 | 0 | 0 |
| 211 | 1 | 0 | 0 |
| 139 | 0 | 0 | 0 |
| 468 | 1 | 1 | 0 |
|  77 | 0 | 0 | 0 |
| 543 | 0 | 0 | 0 |
| 703 | 0 | 0 | 0 |
| 165 | 0 | 1 | 0 |
| 415 | 1 | 0 | 0 |
| 666 | 0 | 0 | 0 |
| 544 | 1 | 1 | 0 |
| 677 | 0 | 0 | 0 |
| 546 | 0 | 1 | 0 |
| 298 | 0 | 0 | 0 |
| 275 | 0 | 0 | 0 |


In [9]:
# build and compile model

model = Sequential([
    Dense (units=16, input_shape=(num_col_x,), activation='relu'),
    Dense (units=32, activation='relu'), 
    Dense (units=num_medic, activation='sigmoid')
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                1792      
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 2,435
Trainable params: 2,435
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [11]:


model.fit(x=X_np, y=y_np,
    batch_size=10, epochs=10, shuffle=True, verbose=2)

Epoch 1/10
64/64 - 1s - loss: 0.4562 - accuracy: 0.3438 - 1s/epoch - 17ms/step
Epoch 2/10
64/64 - 0s - loss: 0.3877 - accuracy: 0.3281 - 174ms/epoch - 3ms/step
Epoch 3/10
64/64 - 0s - loss: 0.3579 - accuracy: 0.3893 - 183ms/epoch - 3ms/step
Epoch 4/10
64/64 - 0s - loss: 0.3311 - accuracy: 0.4192 - 183ms/epoch - 3ms/step
Epoch 5/10
64/64 - 0s - loss: 0.2989 - accuracy: 0.4819 - 182ms/epoch - 3ms/step
Epoch 6/10
64/64 - 0s - loss: 0.2698 - accuracy: 0.4710 - 194ms/epoch - 3ms/step
Epoch 7/10
64/64 - 0s - loss: 0.2379 - accuracy: 0.4867 - 184ms/epoch - 3ms/step
Epoch 8/10
64/64 - 0s - loss: 0.2141 - accuracy: 0.5338 - 170ms/epoch - 3ms/step
Epoch 9/10
64/64 - 0s - loss: 0.1885 - accuracy: 0.5385 - 160ms/epoch - 3ms/step
Epoch 10/10
64/64 - 0s - loss: 0.1629 - accuracy: 0.5228 - 173ms/epoch - 3ms/step


In [12]:
predictions = model.predict(X_val_np)


5/5 [==============================] - 0s 3ms/step


## Handle Result


In [18]:
col_names =[]
for i in  train_y.columns.tolist():
    i +='(predicted)'
    col_names.append(i) 


# df_predictions = probability of "has" that medicine (有該藥材)
df_predictions = pd.DataFrame(predictions, columns=col_names)
print("predictions.shape:", predictions.shape)

yes_thres = 0.5         # threshold for summarized as "has that medicine"

# all_class_binary_prediction_df = predicted as 是否有該藥材
df_binary_prediction_all_class = df_predictions.apply(lambda x: [1 if i > yes_thres else 0 for i in x])

# pred_and_truth = predicted as 是否有該藥材, and ground truth
pred_and_truth = pd.concat([df_binary_prediction_all_class, val_y.iloc[:, :num_medic]], axis=1)

predictions.shape: (159, 3)


C:\Users\taliah\AppData\Local\Temp\ipykernel_2164\4026871775.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_class_binary_prediction_df = df_predictions.applymap(lambda x: 1 if x > yes_thres else 0)


In [14]:
print(df_predictions)

     麻黃(predicted)  桂枝(predicted)  荊芥(predicted)
0     2.958135e-04       0.941749   4.666733e-05
1     7.339049e-03       0.076869   2.740976e-03
2     1.054771e-03       0.000288   1.447271e-05
3     1.668761e-02       0.028680   2.033905e-02
4     3.570214e-10       0.180459   1.491410e-07
..             ...            ...            ...
154   2.307215e-01       0.166336   1.015897e-01
155   2.307215e-01       0.166336   1.015897e-01
156   1.736707e-05       0.326245   2.248381e-03
157   2.017001e-01       0.627275   5.237541e-02
158   9.074086e-05       0.422568   1.622641e-04

[159 rows x 3 columns]


In [19]:

print(df_binary_prediction_all_class)


     麻黃(predicted)  桂枝(predicted)  荊芥(predicted)
0                0              1              0
1                0              0              0
2                0              0              0
3                0              0              0
4                0              0              0
..             ...            ...            ...
154              0              0              0
155              0              0              0
156              0              0              0
157              0              1              0
158              0              0              0

[159 rows x 3 columns]


In [20]:
print(pred_and_truth)

     麻黃(predicted)  桂枝(predicted)  荊芥(predicted)   麻黃   桂枝   荊芥
0              0.0            1.0            0.0  NaN  NaN  NaN
1              0.0            0.0            0.0  NaN  NaN  NaN
2              0.0            0.0            0.0  NaN  NaN  NaN
3              0.0            0.0            0.0  NaN  NaN  NaN
4              0.0            0.0            0.0  0.0  1.0  0.0
..             ...            ...            ...  ...  ...  ...
781            NaN            NaN            NaN  0.0  0.0  0.0
694            NaN            NaN            NaN  0.0  0.0  0.0
448            NaN            NaN            NaN  0.0  1.0  1.0
632            NaN            NaN            NaN  0.0  0.0  0.0
274            NaN            NaN            NaN  0.0  0.0  0.0

[283 rows x 6 columns]


In [17]:
from utility_file import my_utilities as myutil

folder_name = "result/train_multi_med_result"
# 
myutil.df_to_csv(pred_and_truth, save_path="./result/multi_med_prediction", file_prefix="pred_and_truth")
myutil.df_to_csv(df_predictions, save_path="./result/multi_med_prediction", file_prefix="probi_each_med")

pred_and_truth saved to ./multi_med_prediction/pred_and_truth.csv
probi_each_med saved to ./multi_med_prediction/probi_each_med.csv


In [ ]:
# calculate true positive, true negative, false positive, false negative and f1 score against the model and val_y?